# GSM8K Math Problem Training

This notebook implements training on the GSM8K dataset using TPU acceleration and optimized configurations for mathematical reasoning.

## Repository Setup

First, let's clone the repository and prepare the environment.

In [ ]:
# Clone the repository and change directory
!git clone https://github.com/VishwamAI/VishwamAI
%cd VishwamAI

In [ ]:
# Install dependencies
!pip install -q --upgrade transformers datasets accelerate bitsandbytes sentencepiece \
    flax optax omegaconf huggingface-hub einops aim>=3.17.5 safetensors
